In [29]:
import numpy as np
import warnings
import time
import sklearn.metrics as skmetrics
from sklearn.calibration import calibration_curve
import plotly.graph_objects as go

warnings.filterwarnings(action='ignore')

In [4]:
class Optimizer(object):

    def __init__(self, alpha, eta_decay_factor, optimizer_type):
        '''
        Arg(s):
            alpha : float
                initial learning rate 
            eta_decay_factor : float
                learning rate decay rate
            optimizer_type : str
                'gradient_descent',
                'stochastic_gradient_descent',
        '''

        self.__alpha = alpha
        self.__eta_decay_factor = eta_decay_factor
        self.__optimizer_type = optimizer_type

    def __compute_gradients(self, w, x, y, loss_func='logistic'):
        '''
        Returns the gradient of a loss function

        Arg(s):
            w : numpy[float32]
                d x 1 weight vector
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss by default is 'logistic' only for the purpose of the assignment
        Returns:
            numpy[float32] : d x 1 gradients
        '''

        if loss_func == 'logistic':
            p = 1 / (1 + np.exp(-1 * y * np.matmul(w.T, x)))
            return -1 * np.mean(y * x * (1 - p), axis=1).reshape(w.shape)
        else:
            raise ValueError('Unupported loss function: {}'.format(loss_func))

    def __polynomial_decay(self, time_step):
        '''
        Computes the polynomial decay factor t^{-a}

        Arg(s):
            time_step : int
                current step in optimization
        Returns:
            float : polynomial decay to adjust learning rate
        '''

        return np.power(1 / time_step, (self.__eta_decay_factor))

    def update(self,
               w,
               x,
               y,
               loss_func,
               batch_size,
               time_step):
        '''
        Updates the weight vector based on

        Arg(s):
            w : numpy[float32]
                d x 1 weight vector
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss function to use, should be 'logistic' for the purpose of the assignment
            batch_size : int
                batch size for stochastic and momentum stochastic gradient descent
            time_step : int
                current step in optimization
        Returns:
            numpy[float32]: d x 1 weights
        '''

        if self.__optimizer_type == 'gradient_descent':
            return w - self.__alpha * self.__compute_gradients(w, x, y, loss_func)
        elif self.__optimizer_type == 'stochastic_gradient_descent':
            batch_idx = np.random.choice(x.shape[1], batch_size)
            return w - self.__polynomial_decay(time_step) * self.__alpha * self.__compute_gradients(w, x[:,batch_idx], y[batch_idx], loss_func)
        else:
            raise ValueError('Unsupported optimizer type: {}'.format(self.__optimizer_type))


In [5]:
class LogisticRegression(object):

    def __init__(self):
        # Define private variables
        self.__weights = None
        self.__optimizer = None

    def fit(self,
            x,
            y,
            T,
            alpha,
            eta_decay_factor,
            batch_size,
            optimizer_type,
            loss_func='logistic'):
        '''
        Fits the model to x and y by updating the weight vector
        using gradient descent

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            T : int
                number of iterations to train
            alpha : float
                learning rate
            eta_decay_factor : float
                learning rate decay rate
            batch_size : int
                number of examples per batch
            optimizer_type : str
                'gradient_descent',
                'momentum_gradient_descent',
                'stochastic_gradient_descent',
                'momentum_stochastic_gradient_descent'
            loss_func : str
                loss function to use, by default is 'logistic' only for the purpose of the assignment
        '''

        self.__optimizer = Optimizer(alpha, eta_decay_factor, optimizer_type)

        self.__weights = np.zeros([x.shape[0], 1])

        for t in range(1, T + 1):

            loss = self.__compute_loss(x, y, loss_func)
            if (t % 100) == 0:
                print('Step={}  Loss={}'.format(t, loss))

            self.__weights = self.__optimizer.update(self.__weights, x, y, loss_func, batch_size, t)

    def predict(self, x):
        '''
        Predicts the label for each feature vector x

        Arg(s):
            x : numpy[float32]
                d x N feature vector
        Returns:
            numpy[float32] : 1 x N vector
        '''

        predictions = 1 / (1 + np.exp(-1 * np.matmul(self.__weights.T, x)))
        return predictions.squeeze()

    def __compute_loss(self, x, y, loss_func):
        '''
        Computes the logistic loss

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss function to use, by default is 'logistic' only for the purpose of the assignment
        Returns:
            float : loss
        '''

        if loss_func == 'logistic':
            loss = np.mean(np.log(1 + np.exp(-1 * y * np.matmul(self.__weights.T, x))))
        else:
            raise ValueError('Unsupported loss function: {}'.format(loss_func))

        return loss

In [9]:
data = np.vectorize(float)(np.load('data/shots.npy', allow_pickle=True))

sbxg = data[:, 0]
y = data[:, 1]
x = data[:, 2:]

shuffled_indices = np.random.permutation(x.shape[0])

train_split_idx = int(0.60 * x.shape[0])
val_split_idx = int(0.80 * x.shape[0])

train_indices = shuffled_indices[0:train_split_idx]
val_indices = shuffled_indices[train_split_idx:val_split_idx]
test_indices = shuffled_indices[val_split_idx:]

x_train, y_train, sbxg_train = x[train_indices, :], y[train_indices], sbxg[train_indices]
x_val, y_val, sbxg_val = x[val_indices, :], y[val_indices], sbxg[val_indices]
x_test, y_test, sbxg_test = x[test_indices, :], y[test_indices], sbxg[test_indices]

x_train = np.transpose(x_train, axes=(1, 0))
x_val = np.transpose(x_val, axes=(1, 0))
x_test = np.transpose(x_test, axes=(1, 0))
y_train = np.expand_dims(y_train, axis=0)
y_val = np.expand_dims(y_val, axis=0)
y_test = np.expand_dims(y_test, axis=0)

y_train = y_train[0]
y_val = y_val[0]
y_test = y_test[0]

In [23]:
model_ours = LogisticRegression()

time_start = time.time()

model_ours.fit(x_train, 
               y_train, 
               T=5000, 
               alpha=1.3e-4, 
               eta_decay_factor=5e-3, 
               batch_size=8000, 
               optimizer_type='stochastic_gradient_descent')

time_elapsed = time.time() - time_start
print('Total training time: {:3f} seconds'.format(time_elapsed))

predictions_train = model_ours.predict(x_train)
loss_score = skmetrics.log_loss(y_train, predictions_train)
print('Training Log-loss of Our Model: {:.4f}'.format(loss_score))

loss_score = skmetrics.log_loss(y_train, sbxg_train)
print('Training Log-loss of StatsBomb Model: {:.4f}'.format(loss_score))

predictions_val = model_ours.predict(x_val)
loss_score = skmetrics.log_loss(y_val, predictions_val)
print('Validation Log-loss of Our Model: {:.4f}'.format(loss_score))

loss_score = skmetrics.log_loss(y_val, sbxg_val)
print('Validation Log-loss of StatsBomb Model: {:.4f}'.format(loss_score))

predictions_test = model_ours.predict(x_test)
loss_score = skmetrics.log_loss(y_test, predictions_test)
print('Testing Log-loss of Our Model: {:.4f}'.format(loss_score))

loss_score = skmetrics.log_loss(y_test, sbxg_test)
print('Testing Log-loss of StatsBomb Model: {:.4f}'.format(loss_score))

Step=100  Loss=0.32023391574942267
Step=200  Loss=0.31773855567747133
Step=300  Loss=0.3157302252020914
Step=400  Loss=0.3141919884554619
Step=500  Loss=0.3130771536282461
Step=600  Loss=0.31218260903672357
Step=700  Loss=0.310494748327992
Step=800  Loss=0.309470166981278
Step=900  Loss=0.30916902223380127
Step=1000  Loss=0.3084053477881781
Step=1100  Loss=0.30688559088110173
Step=1200  Loss=0.3061559388474312
Step=1300  Loss=0.3061363353092344
Step=1400  Loss=0.30494622974602287
Step=1500  Loss=0.30475181733166873
Step=1600  Loss=0.30367974850497437
Step=1700  Loss=0.30359770845511463
Step=1800  Loss=0.3028695239503065
Step=1900  Loss=0.30258132772480756
Step=2000  Loss=0.3018162849256347
Step=2100  Loss=0.3018986879296898
Step=2200  Loss=0.30139877974840373
Step=2300  Loss=0.30351341680721045
Step=2400  Loss=0.3009488077174985
Step=2500  Loss=0.3009164863597641
Step=2600  Loss=0.3007163956486043
Step=2700  Loss=0.29994303658148697
Step=2800  Loss=0.2996873645788858
Step=2900  Loss=0.

In [53]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')

prob_true, prob_pred = calibration_curve(y_test, predictions_test, n_bins=10, strategy='quantile')
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='Our Model'))

prob_true, prob_pred = calibration_curve(y_test, sbxg_test, n_bins=10, strategy='quantile')
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='StatsBomb Model'))

fig.add_trace(go.Scatter(x=np.linspace(0, 1, 10), y=np.linspace(0, 1, 10), mode='lines', line=dict(dash='dash'), name='Perfect Calibration'))

fig.update_layout(
    title='Calibration Curves with Quantile Buckets',
    xaxis_title='Mean xG',
    yaxis_title='Fraction of Actual Goals',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1,
        xanchor='right',
        x=1
    ),
    width=600,
    height=600
)

fig.show()

In [84]:
fig = go.Figure()
fig.update_layout(template='plotly_dark')

prob_true, prob_pred = calibration_curve(y_test, predictions_test, n_bins=10)
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='Our Model'))

prob_true, prob_pred = calibration_curve(y_test, sbxg_test, n_bins=10)
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode='markers+lines', name='StatsBomb'))

fig.add_trace(go.Scatter(x=np.linspace(0, 1, 10), y=np.linspace(0, 1, 10), mode='lines', line=dict(dash='dash'), name='Perfect Calibration'))

predictions_test_binned = np.digitize(predictions_test, bins=np.linspace(0, 1, 11)) - 1
bin_labels = [f"{(i / 10)}" for i in range(1,11)]
bin_counts = np.bincount(predictions_test_binned, minlength=10)

fig.add_trace(go.Bar(x=bin_labels, y=bin_counts, name='Number of Shots per Bucket', yaxis='y2', xaxis='x2', opacity=0.5))

fig.update_layout(
    title='Calibration Curves with Uniform Buckets',
    xaxis_title='Mean xG',
    yaxis={
        'range': [0, 1],
        'title': 'Fraction of Actual Goals'
    },
    yaxis2={
        'title': 'Number of Shots',
        'overlaying': 'y',
        'side': 'right',
        'showgrid':False,
    },
    xaxis2={
        'overlaying': 'x',
        'side': 'top',
        'showgrid':False,
        'showticklabels':False
    },
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1,
        xanchor='right',
        x=1
    ),
    width=600,
    height=600
)

fig.show()